In [1]:
! git clone https://github.com/IlyaGusev/summarus.git

Cloning into 'summarus'...
remote: Enumerating objects: 1484, done.
remote: Counting objects: 100% (118/118), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 1484 (delta 99), reused 96 (delta 79), pack-reused 1366
Receiving objects: 100% (1484/1484), 504.99 KiB | 3.16 MiB/s, done.
Resolving deltas: 100% (988/988), done.


## Подготовка данных

In [5]:
import os
import sys
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from datasets import Dataset, DatasetDict
import razdel
import fasttext

from evaluate import print_metrics, postprocess

In [10]:
%ls /DATA/ichuviliaeva/project_data

articles_part_0.csv  articles_part_4.csv  articles_part_8.csv  val_data.json
articles_part_1.csv  articles_part_5.csv  query_list.txt       visited_set.txt
articles_part_2.csv  articles_part_6.csv  test_data.json
articles_part_3.csv  articles_part_7.csv  train_data.json


In [12]:
! wget https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.ftz
lang_detector = fasttext.load_model('lid.176.ftz')

data_path = '/DATA/ichuviliaeva/project_data'
data = []
for file_name in os.listdir(data_path):
    if 'articles' not in file_name:
        continue
    data_part = pd.read_csv(os.path.join(data_path, file_name))

    # Checking that article and abstract are written in russian
    for article, abstract in tqdm(data_part[['article', 'abstract']].to_numpy()):
        if lang_detector.predict(article.replace('\n', ' '), k=1)[0][0] == '__label__en' or \
            lang_detector.predict(abstract.replace('\n', ' '), k=1)[0][0] == '__label__en':
            continue
        data.append([article, abstract])

data = pd.DataFrame(data, columns=['article', 'abstract'])

data.head()

--2023-04-10 01:27:16--  https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.ftz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 108.156.22.14, 108.156.22.118, 108.156.22.105, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|108.156.22.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 938013 (916K) [binary/octet-stream]
Saving to: ‘lid.176.ftz.1’

lid.176.ftz.1       100%[===================>] 916.03K  --.-KB/s    in 0.1s    

2023-04-10 01:27:17 (6.36 MB/s) - ‘lid.176.ftz.1’ saved [938013/938013]



  0%|          | 0/800 [00:00<?, ?it/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

...

  0%|          | 0/5000 [00:00<?, ?it/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

,article,abstract
0,Состав и калорийность\nПлоды айвы достаточно н...,"Айву, которую часто называют ложным яблоком вс..."
1,В последние годы при госпитализации больных с ...,Исследовалась частота ассоциации заболеваний щ...
2,"Введение. Одной из серьезных проблем, стоящих ...",Цель исследования - провести анализ динамики с...
3,Актуальность проблемы. На современном этапе пр...,В статье проанализированы данные о впервые выя...
4,Сохранение здоровья детей в Российской Федерац...,В статье представлены краткие данные о распрос...


In [13]:
display(data)

,article,abstract
0,Состав и калорийность\nПлоды айвы достаточно н...,"Айву, которую часто называют ложным яблоком вс..."
1,В последние годы при госпитализации больных с ...,Исследовалась частота ассоциации заболеваний щ...
2,"Введение. Одной из серьезных проблем, стоящих ...",Цель исследования - провести анализ динамики с...
3,Актуальность проблемы. На современном этапе пр...,В статье проанализированы данные о впервые выя...
4,Сохранение здоровья детей в Российской Федерац...,В статье представлены краткие данные о распрос...
...,...,...
38670,Одним из значимых симптомов в структуре дефект...,Работа посвящена рассмотрению проблемы коррекц...
38671,Актуальность обращения к методу сенсорной инте...,Представлены результаты экспериментального исс...
38672,"По определению, принятому в научной литературе...",В статье рассматриваются условия успешного обу...
38673,В настоящее время в логопедии понятие «звуко-с...,В статье рассматривается актуальная проблема д...


In [11]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size=0.2, random_state=42)
test, val = train_test_split(test, test_size=0.5, random_state=42)

In [12]:
display(val)

,article,abstract
2730,На фоне таких по-настоящему крупных церковных ...,Дается характеристика жизни и творческого насл...
29656,Введение\nАтомные станции (АС) являются потенц...,Рассматриваются трехмерные нелинейные задачи д...
11325,"Введение\nУстановлено, что печень, как правило...",Введение мерказолила (25 мг/кг 20 дней) per se...
10755,"Значительное изменение социально-политических,...",Статья посвящена территориальным градостроител...
10530,Австрийская Республика приняла статус постоянн...,Рассматривается вопрос трансформации постоянно...
...,...,...
28612,Аграрный вестник Урала\n87\nЛесное хозяйство\n...,На основании отчётных данных за период с 1980 ...
22237,"слепых; система спортивной подготов- athletes,...",В статье описан опыт применения комплексного к...
15315,анализ.\nВведение. В основе подходов к упрочне...,В работе рассматривается и формируется нелиней...
17639,"underground water, swamp water, chemistry, tai...",Проведено обобщение данных по химическому сост...


In [14]:
dataset = Dataset.from_pandas(data)
dataset = dataset.train_test_split(test_size=0.2, seed=42)
test_val_dataset = dataset['test'].train_test_split(test_size=0.5, seed=42)

datasets = DatasetDict({
    'train': dataset['train'],
    'valid': test_val_dataset['train'],
    'test': test_val_dataset['test']
})
datasets

DatasetDict({
    train: Dataset({
        features: ['article', 'abstract'],
        num_rows: 30940
    })
    valid: Dataset({
        features: ['article', 'abstract'],
        num_rows: 3867
    })
    test: Dataset({
        features: ['article', 'abstract'],
        num_rows: 3868
    })
})

In [15]:
datasets['train'].to_json('/DATA/ichuviliaeva/project_data/train_data.json')
datasets['valid'].to_json('/DATA/ichuviliaeva/project_data/val_data.json')
datasets['test'].to_json('/DATA/ichuviliaeva/project_data/test_data.json')

Creating json from Arrow format:   0%|          | 0/31 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

367068122

## Bart

In [1]:
%ls

bart_preprocess.sh  ru_articles_summarization/             summarus/
fairseq/            ru-articles-summarization-mbart.ipynb


In [2]:
%cd summarus

/home/ichuviliaeva/is_project/summarus


In [2]:
import os

In [3]:
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [5]:
import json
import torch
from tqdm.notebook import tqdm
from transformers import MBartTokenizer, MBartForConditionalGeneration


def gen_batch(inputs, batch_size):
    batch_start = 0
    while batch_start < len(inputs):
        yield inputs[batch_start: batch_start + batch_size]
        batch_start += batch_size

def predict(
    model_name,
    test_file,
    predictions_file,
    targets_file,
    max_source_tokens_count=600,
    max_target_tokens_count=160,
    use_cuda=True,
    batch_size=4
):
    inputs = []
    targets = []
    
    with open(test_file, "r") as r:
        for line in r:
            record = json.loads(line)
            inputs.append(record["article"])
            targets.append(record["abstract"].replace("\n", " "))
        
    tokenizer = MBartTokenizer.from_pretrained(model_name)
    device = torch.device("cuda:0") if use_cuda else torch.device("cpu")
    model = MBartForConditionalGeneration.from_pretrained(model_name).to(device)
    
    predictions = []
    batches = gen_batch(inputs, batch_size)
    for batch in tqdm(batches):
        input_ids = tokenizer.prepare_seq2seq_batch(
            batch,
            src_lang="ru_RU",
            return_tensors="pt",
            padding="max_length",
            truncation=True,
            max_length=max_source_tokens_count
        )["input_ids"].to(device)
        output_ids = model.generate(
            input_ids=input_ids,
            max_length=max_target_tokens_count + 2,
            no_repeat_ngram_size=3,
            num_beams=5,
            top_k=0
        )
        summaries = tokenizer.batch_decode(output_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
        predictions.extend(summaries)
        
    with open(predictions_file, "w") as w:
        for p in predictions:
            w.write(p.strip() + "\n")
    with open(targets_file, "w") as w:
        for t in targets:
            w.write(t.strip() + "\n")
        
    return predictions, targets

In [6]:
predictions, targets = predict(
    "IlyaGusev/mbart_ru_sum_gazeta", 
    "/DATA/ichuviliaeva/project_data/test_data.json",
    "/DATA/ichuviliaeva/project_data/predictions.txt", 
    "/DATA/ichuviliaeva/project_data/targets.txt", 
    batch_size = 16
)

0it [00:00, ?it/s]

/home/ichuviliaeva/miniconda3/envs/is_project/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:3712: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


In [ ]:
# работает около часа

In [10]:
import os
import argparse
import re

import numpy as np

import razdel
import nltk

from collections import Counter
from statistics import mean

from rouge import Rouge
from nltk.translate.bleu_score import corpus_bleu
from nltk.translate.chrf_score import corpus_chrf
import torch


def calc_metrics(refs, hyps):
    metrics = dict()
    metrics["count"] = len(hyps)
    metrics["ref_example"] = refs[-1]
    metrics["hyp_example"] = hyps[-1]
    many_refs = [[r] if r is not list else r for r in refs]

    # Calculate BLEU score
    t_hyps = [hyp.split(" ") for hyp in hyps]
    t_refs = [[r.split(" ") for r in rs] for rs in many_refs]
    metrics["bleu"] = corpus_bleu(t_refs, t_hyps)

    # Calculate BLEU score
    rouge = Rouge()
    # Remove empty hypotheses
    idxs = np.where([len(h) > 0 for h in hyps])[0]
    scores = rouge.get_scores(np.array(hyps)[idxs], np.array(refs)[idxs], avg=True)
    metrics.update(scores)

    # Calculate mean abstracts' length
    metrics["length"] = mean([len(h) for h in hyps])

    return metrics


def print_metrics(refs, hyps):
    metrics = calc_metrics(refs, hyps)

    print("-------------METRICS-------------")
    print("Count:\t", metrics["count"])
    print("Ref:\t", metrics["ref_example"])
    print("Hyp:\t", metrics["hyp_example"])

    print("BLEU:     \t{:3.1f}".format(metrics["bleu"] * 100.0))

    print("ROUGE-1-F:\t{:3.1f}".format(metrics["rouge-1"]['f'] * 100.0))
    print("ROUGE-2-F:\t{:3.1f}".format(metrics["rouge-2"]['f'] * 100.0))
    print("ROUGE-L-F:\t{:3.1f}".format(metrics["rouge-l"]['f'] * 100.0))

    print("Avg length:\t{:3.1f}".format(metrics["length"]))


def postprocess(ref, hyp, tokenize_after=False, lower=False):
    ref = ref.strip()
    hyp = hyp.strip()
    if tokenize_after:
        hyp = hyp.replace("@@UNKNOWN@@", "<unk>")
        hyp = " ".join([token.text for token in razdel.tokenize(hyp)])
        ref = " ".join([token.text for token in razdel.tokenize(ref)])
    if lower:
        hyp = hyp.lower()
        ref = ref.lower()
    return ref, hyp

In [33]:
from tqdm.notebook import tqdm

def calcs(predictions_file, targets_file):
    preds = []
    targets = []
    with open(predictions_file, "r") as r:
        for rl in tqdm(r.readlines()):
            preds.append(rl)
    with open(targets_file, "r") as r:
        for rl in tqdm(r.readlines()):
            targets.append(rl)
            
    for i, (target, pred) in enumerate(tqdm(zip(targets, preds))):
        targets[i], preds[i] = postprocess(target, pred, tokenize_after=True, lower=True)
        
    print_metrics(targets, preds)
    

In [34]:
calcs("/DATA/ichuviliaeva/project_data/predictions.txt", 
    "/DATA/ichuviliaeva/project_data/targets.txt")

  0%|          | 0/3868 [00:00<?, ?it/s]

  0%|          | 0/3868 [00:00<?, ?it/s]

0it [00:00, ?it/s]

-------------METRICS-------------
Count:	 3868
Ref:	 исследована возможность получения кисломолочных продуктов с бифидобактериями на основе биогеля из водорослей . показана его пребиотическая активность в отношении bifidobacterium bifidum . разработана технология получения пробиотических продуктов , установлен срок хранения .
Hyp:	 наибольший интерес в связи с этим представляют пробиотические продукты на основе бифидобактерий — главной составляющей облигатной микрофлоры кишечника человека . они плохо растут на молоке , что создает определенные трудности в производстве кисломолочных продуктов .
BLEU:     	2.6
ROUGE-1-F:	17.9
ROUGE-2-F:	6.2
ROUGE-L-F:	16.0
Avg length:	328.9
